In [1]:
import pandas as pd

# READ DATASETS

In [3]:
folder = "./datasets/"


### STATIONS DATASET


In [6]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [7]:

df = pd.read_csv(folder + "datos_temp.csv", header=0).drop(['Unnamed: 0'], axis=1)

In [10]:
df.head()

,Unnamed: 0,id,codi_estacio,codi_variable,data_lectura,valor_lectura
0,0,CA320101090000,CA,32,2009-01-01T00:00:00.000,6.0
1,1,CC320101090000,CC,32,2009-01-01T00:00:00.000,4.6
2,2,CD320101090000,CD,32,2009-01-01T00:00:00.000,3.4
3,3,CE320101090000,CE,32,2009-01-01T00:00:00.000,7.3
4,4,CG320101090000,CG,32,2009-01-01T00:00:00.000,5.0


# PROCESS DATA

In [28]:
codiE_por_comarca = df_stations.groupby('comarca')['codiE'].unique().reset_index()
codiE_x_comarca = codiE_por_comarca.to_dict(orient='records')

In [30]:
codiE_por_comarca

,comarca,codiE
0,Alt Camp,"[VY, DA, DQ]"
1,Alt Empordà,"[U1, W1, J5, VZ, Y5, D6, D4, U2, W2, XZ, W3]"
2,Alt Penedès,"[DI, W4, WP, U3, WY, YO]"
3,Alt Urgell,"[Y4, CD, W5, CJ]"
4,Alta Ribagorça,"[CT, Z2]"
5,Anoia,"[CE, XB, XA, H1, YH]"
6,Bages,"[WW, MQ, U4, R1, WN, CL]"
7,Baix Camp,"[DE, YF, U5, XR, YL, W6, U6]"
8,Baix Ebre,"[U7, U8, DB, U9, UA, X5]"
9,Baix Empordà,"[DO, UC, DF, UB, YP, UD, UE]"
